# Scenario 06: Moderated Agent Discussions

**Estimated Time**: 45 minutes

## Learning Objectives
- Implement moderated multi-agent discussions
- Handle turn-taking and conflict resolution
- Build debate and round-robin protocols
- Synthesize conclusions from multiple perspectives

## Prerequisites
- Completed Scenario 01 (Simple Agent + MCP)
- Understanding of agent coordination patterns

In [1]:
# Load environment and configure paths
import sys
from pathlib import Path

# Add project root to path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

print(f"✅ Project root: {project_root}")

✅ Project root: C:\Users\jonasrotter\OneDrive - Microsoft\Desktop\Jonas Privat\MyCodingProjects\agents-workshop


In [2]:
# Setup and imports
import sys
from pathlib import Path
import asyncio

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.agents.moderator_agent import (
    ModeratorAgent,
    DiscussionPhase,
    DiscussionTurn,
    DiscussionSummary,
    ConflictStrategy,
)
from src.agents.discussion import (
    DiscussionProtocol,
    DiscussionConfig,
    DiscussionRole,
    Participant,
    DebateProtocol,
    RoundRobinProtocol,
    create_debate,
    create_roundtable,
)

print("✅ Imports successful")

✅ Imports successful


## Part 1: Creating Mock Agents for Discussion

First, let's create some mock agents with different perspectives.

In [3]:
# Create mock agents for demonstration
class MockDebateAgent:
    """Mock agent for discussion demonstrations."""
    
    def __init__(self, name: str, perspective: str):
        self.name = name
        self.perspective = perspective
        self._turn_count = 0
    
    async def run(self, prompt: str) -> str:
        self._turn_count += 1
        
        # Generate response based on perspective
        if "opening" in prompt.lower():
            return f"""As {self.name}, I believe {self.perspective}. 
This is my opening statement on the topic.
Key argument: The evidence clearly supports my position."""
        
        elif "rebuttal" in prompt.lower():
            return f"""{self.name} disagrees with the previous points.
My counter-argument is based on {self.perspective}.
The opposing view fails to consider important factors."""
        
        else:
            return f"""{self.name} (Turn {self._turn_count}): 
Building on my {self.perspective} perspective,
I would like to add that we should consider the broader implications.
This aligns with current trends and research."""

# Create agents with different perspectives
optimist = MockDebateAgent("Optimist", "technology will solve our problems")
skeptic = MockDebateAgent("Skeptic", "we should be cautious about AI risks")
pragmatist = MockDebateAgent("Pragmatist", "we need balanced regulation")

print("Created agents:")
print(f"  - {optimist.name}: {optimist.perspective}")
print(f"  - {skeptic.name}: {skeptic.perspective}")
print(f"  - {pragmatist.name}: {pragmatist.perspective}")

Created agents:
  - Optimist: technology will solve our problems
  - Skeptic: we should be cautious about AI risks
  - Pragmatist: we need balanced regulation


## Part 2: The Moderator Agent

The `ModeratorAgent` facilitates discussions between multiple agents:
- Manages turn-taking
- Detects conflicts
- Synthesizes conclusions

In [4]:
# Create a moderator
moderator = ModeratorAgent(
    name="moderator",
    conflict_strategy=ConflictStrategy.EXPLORE,
    max_rounds=5,
)

# Register participants
moderator.register_participant(optimist)
moderator.register_participant(skeptic)
moderator.register_participant(pragmatist)

print(f"Moderator: {moderator.name}")
print(f"Conflict strategy: {moderator.conflict_strategy}")
print(f"Max rounds: {moderator.max_rounds}")
print(f"Participants: {moderator.participants}")

Moderator: moderator
Conflict strategy: ConflictStrategy.EXPLORE
Max rounds: 5
Participants: ['Optimist', 'Skeptic', 'Pragmatist']


In [5]:
# Start a discussion
async def run_moderated_discussion():
    topic = "Should AI development be regulated by governments?"
    
    print(f"\n📋 Discussion Topic: {topic}\n")
    print("=" * 60)
    
    # Opening statements
    await moderator.start_discussion(topic)
    
    print("\n🎤 Opening Statements:\n")
    for turn in moderator.turn_history:
        if turn.phase == DiscussionPhase.OPENING:
            print(f"{turn.participant}:")
            print(f"  {turn.content[:200]}...\n")
    
    # Run discussion rounds
    for round_num in range(1, 3):
        print(f"\n🔄 Round {round_num}:\n")
        turns = await moderator.run_round()
        for turn in turns:
            print(f"{turn.participant}:")
            print(f"  {turn.content[:150]}...\n")
    
    # Synthesize conclusion
    print("\n📊 Synthesis:\n")
    summary = await moderator.synthesize()
    
    print(f"Conclusion: {summary.conclusion[:300]}...")
    print(f"\nKey Points: {summary.key_points}")
    print(f"Consensus Reached: {summary.consensus_reached}")
    
    return summary

# Run the discussion
import nest_asyncio
nest_asyncio.apply()

import asyncio
summary = asyncio.get_event_loop().run_until_complete(run_moderated_discussion())


📋 Discussion Topic: Should AI development be regulated by governments?


🎤 Opening Statements:

Optimist:
  As Optimist, I believe technology will solve our problems. 
This is my opening statement on the topic.
Key argument: The evidence clearly supports my position....

Skeptic:
  As Skeptic, I believe we should be cautious about AI risks. 
This is my opening statement on the topic.
Key argument: The evidence clearly supports my position....

Pragmatist:
  As Pragmatist, I believe we need balanced regulation. 
This is my opening statement on the topic.
Key argument: The evidence clearly supports my position....


🔄 Round 1:

Optimist:
  As Optimist, I believe technology will solve our problems. 
This is my opening statement on the topic.
Key argument: The evidence clearly supports my ...

Skeptic:
  As Skeptic, I believe we should be cautious about AI risks. 
This is my opening statement on the topic.
Key argument: The evidence clearly supports my...

Pragmatist:
  As Pragmatist, I b

## Part 3: Discussion Protocols

Different protocols provide different discussion structures:
- **DiscussionProtocol**: General-purpose
- **DebateProtocol**: Formal pro/con debate
- **RoundRobinProtocol**: Each participant speaks in turn

In [6]:
# Create a debate protocol
debate_config = DiscussionConfig(
    topic="Is open-source AI safer than closed-source AI?",
    max_rounds=7,
    conflict_strategy=ConflictStrategy.EXPLORE,
    allow_rebuttals=True,
    rebuttal_rounds=1,
)

protocol = DiscussionProtocol(debate_config)

# Register participants with roles
protocol.register_participant(
    optimist, 
    role=DiscussionRole.PROPONENT, 
    priority=10
)
protocol.register_participant(
    skeptic, 
    role=DiscussionRole.OPPONENT, 
    priority=10
)
protocol.register_participant(
    pragmatist, 
    role=DiscussionRole.NEUTRAL, 
    priority=5
)

print("Discussion Configuration:")
print(f"  Topic: {debate_config.topic}")
print(f"  Max Rounds: {debate_config.max_rounds}")
print(f"  Rebuttals: {debate_config.allow_rebuttals}")
print(f"\nParticipants:")
for p in protocol.participants:
    print(f"  - {p.name} ({p.role.value})")

Discussion Configuration:
  Topic: Is open-source AI safer than closed-source AI?
  Max Rounds: 7
  Rebuttals: True

Participants:
  - Optimist (proponent)
  - Skeptic (opponent)
  - Pragmatist (neutral)


In [7]:
# Add event callbacks
def on_turn(turn: DiscussionTurn):
    print(f"  📢 {turn.participant}: {turn.content[:80]}...")

def on_round(result):
    print(f"\n  ⏱️ Round {result.round_number} completed")
    print(f"     Turns: {len(result.turns)}, Duration: {result.duration_seconds:.2f}s")

protocol.on_turn(on_turn)
protocol.on_round(on_round)

print("Event callbacks registered")

Event callbacks registered


In [8]:
# Run the full discussion
async def run_protocol_discussion():
    print(f"\n🎯 Starting Discussion: {protocol.config.topic}\n")
    print("=" * 60)
    
    summary = await protocol.run_discussion()
    
    print("\n" + "=" * 60)
    print("\n📊 Discussion Summary:")
    print(f"  Total Rounds: {summary.total_rounds}")
    print(f"  Total Turns: {summary.total_turns}")
    print(f"  Conflicts: {len(summary.conflicts)}")
    print(f"  Consensus: {summary.consensus_reached}")
    
    return summary

# Need fresh agents for a new discussion
optimist = MockDebateAgent("Optimist", "technology will solve our problems")
skeptic = MockDebateAgent("Skeptic", "we should be cautious about AI risks")
pragmatist = MockDebateAgent("Pragmatist", "we need balanced regulation")

# Create fresh protocol
protocol = DiscussionProtocol(debate_config)
protocol.register_participant(optimist, role=DiscussionRole.PROPONENT)
protocol.register_participant(skeptic, role=DiscussionRole.OPPONENT)
protocol.register_participant(pragmatist, role=DiscussionRole.NEUTRAL)
protocol.on_turn(on_turn)
protocol.on_round(on_round)

summary = asyncio.get_event_loop().run_until_complete(run_protocol_discussion())


🎯 Starting Discussion: Is open-source AI safer than closed-source AI?

  📢 Optimist: As Optimist, I believe technology will solve our problems. 
This is my opening s...
  📢 Skeptic: As Skeptic, I believe we should be cautious about AI risks. 
This is my opening ...
  📢 Pragmatist: As Pragmatist, I believe we need balanced regulation. 
This is my opening statem...

  ⏱️ Round 1 completed
     Turns: 3, Duration: 0.00s
  📢 Optimist: As Optimist, I believe technology will solve our problems. 
This is my opening s...
  📢 Skeptic: As Skeptic, I believe we should be cautious about AI risks. 
This is my opening ...
  📢 Pragmatist: As Pragmatist, I believe we need balanced regulation. 
This is my opening statem...

  ⏱️ Round 2 completed
     Turns: 3, Duration: 0.00s
  📢 Optimist: As Optimist, I believe technology will solve our problems. 
This is my opening s...
  📢 Skeptic: As Skeptic, I believe we should be cautious about AI risks. 
This is my opening ...
  📢 Pragmatist: As Pragmatist, I 

## Part 4: Formal Debate Protocol

The `DebateProtocol` provides a structured format for pro/con debates.

In [9]:
# Create a formal debate
proponent = MockDebateAgent("ProAI", "AI will benefit humanity")
opponent = MockDebateAgent("CautiousAI", "AI poses existential risks")
judge = MockDebateAgent("Judge", "evaluating arguments objectively")

debate = create_debate(
    topic="AI systems should have legal personhood",
    proponent=proponent,
    opponent=opponent,
    judges=[judge],
)

print("Formal Debate Setup:")
print(f"  Topic: AI systems should have legal personhood")
print(f"  Proponent: {proponent.name}")
print(f"  Opponent: {opponent.name}")
print(f"  Judge: {judge.name}")

Formal Debate Setup:
  Topic: AI systems should have legal personhood
  Proponent: ProAI
  Opponent: CautiousAI
  Judge: Judge


## Part 5: Round-Robin Discussion

The `RoundRobinProtocol` ensures each participant speaks in order.

In [10]:
# Create agents for roundtable
experts = [
    MockDebateAgent("Economist", "economic growth potential"),
    MockDebateAgent("Ethicist", "moral implications"),
    MockDebateAgent("Engineer", "technical feasibility"),
    MockDebateAgent("Policymaker", "regulatory frameworks"),
]

roundtable = create_roundtable(
    topic="How should society prepare for AGI?",
    participants=experts,
    rounds=2,
)

print("Roundtable Setup:")
print(f"  Topic: How should society prepare for AGI?")
print(f"  Participants: {roundtable.participant_names}")

Roundtable Setup:
  Topic: How should society prepare for AGI?
  Participants: ['Economist', 'Ethicist', 'Engineer', 'Policymaker']


## Part 6: Conflict Detection and Resolution

The moderator detects conflicts and applies resolution strategies.

In [11]:
# Demonstrate conflict strategies
print("Available Conflict Strategies:")
for strategy in ConflictStrategy:
    print(f"  - {strategy.value}: ", end="")
    if strategy == ConflictStrategy.ACKNOWLEDGE:
        print("Note disagreement and move on")
    elif strategy == ConflictStrategy.EXPLORE:
        print("Ask for clarification and deeper discussion")
    elif strategy == ConflictStrategy.VOTE:
        print("Let all participants weigh in")
    elif strategy == ConflictStrategy.DEFER:
        print("Move on, revisit later")

Available Conflict Strategies:
  - acknowledge: Note disagreement and move on
  - explore: Ask for clarification and deeper discussion
  - vote: Let all participants weigh in
  - defer: Move on, revisit later


In [12]:
# Create moderator with different strategy
strict_moderator = ModeratorAgent(
    name="strict_moderator",
    conflict_strategy=ConflictStrategy.VOTE,
    max_rounds=2,
)

print(f"Strict moderator uses: {strict_moderator.conflict_strategy}")

# Create lenient moderator
lenient_moderator = ModeratorAgent(
    name="lenient_moderator",
    conflict_strategy=ConflictStrategy.DEFER,
    max_rounds=5,
)

print(f"Lenient moderator uses: {lenient_moderator.conflict_strategy}")

Strict moderator uses: ConflictStrategy.VOTE
Lenient moderator uses: ConflictStrategy.DEFER


## Part 7: Participant Statistics

Track engagement and cross-references between participants.

In [13]:
# Get participant statistics from previous discussion
stats = protocol.get_participant_stats()

print("\n📈 Participant Statistics:\n")
for name, data in stats.items():
    print(f"{name}:")
    print(f"  Role: {data['role']}")
    print(f"  Total Turns: {data['total_turns']}")
    print(f"  Total Words: {data['total_words']}")
    print(f"  References Made: {data['references_made']}")
    print()


📈 Participant Statistics:

Optimist:
  Role: proponent
  Total Turns: 9
  Total Words: 225
  References Made: 9

Skeptic:
  Role: opponent
  Total Turns: 9
  Total Words: 243
  References Made: 9

Pragmatist:
  Role: neutral
  Total Turns: 9
  Total Words: 216
  References Made: 9



In [14]:
# Get cross-references
cross_refs = protocol.get_cross_references()

print("\n🔗 Cross-References:\n")
for participant, refs in cross_refs.items():
    if refs:
        print(f"{participant} referenced: {refs}")
    else:
        print(f"{participant}: No references to others")


🔗 Cross-References:

Optimist referenced: ['Optimist', 'Optimist', 'Optimist', 'Optimist', 'Optimist', 'Optimist', 'Optimist', 'Optimist', 'Optimist']
Skeptic referenced: ['Skeptic', 'Skeptic', 'Skeptic', 'Skeptic', 'Skeptic', 'Skeptic', 'Skeptic', 'Skeptic', 'Skeptic']
Pragmatist referenced: ['Pragmatist', 'Pragmatist', 'Pragmatist', 'Pragmatist', 'Pragmatist', 'Pragmatist', 'Pragmatist', 'Pragmatist', 'Pragmatist']


## 🎯 Exercise: Add a Debate Participant

Create a new debate participant with a unique perspective:

1. Create a `MockDebateAgent` with a "devil's advocate" perspective
2. Register it with `DiscussionRole.DEVIL_ADVOCATE`
3. Run a short discussion and observe how it affects the debate

In [15]:
# Your solution here

# 1. Create devil's advocate agent
devils_advocate = MockDebateAgent(
    "DevilsAdvocate",
    "challenging all assumptions and questioning everything"
)

# 2. Create a new discussion with the devil's advocate
config = DiscussionConfig(
    topic="Should AI be taught human values?",
    max_rounds=2,
    conflict_strategy=ConflictStrategy.EXPLORE,
)

new_protocol = DiscussionProtocol(config)

# Register participants including devil's advocate
new_protocol.register_participant(
    MockDebateAgent("Believer", "AI alignment is essential"),
    role=DiscussionRole.PROPONENT,
)
new_protocol.register_participant(
    devils_advocate,
    role=DiscussionRole.DEVIL_ADVOCATE,
    priority=15,  # High priority to speak early
)
new_protocol.register_participant(
    MockDebateAgent("Moderate", "balanced approach"),
    role=DiscussionRole.NEUTRAL,
)

print("New discussion participants:")
for p in new_protocol.participants:
    print(f"  - {p.name} ({p.role.value}, priority={p.priority})")

New discussion participants:
  - Believer (proponent, priority=0)
  - DevilsAdvocate (devil_advocate, priority=15)
  - Moderate (neutral, priority=0)


## Summary

In this scenario, you learned:

1. **Moderator Agent**: Facilitates turn-taking and manages discussions
2. **Discussion Protocols**: Different structures for different needs
3. **Participant Roles**: Proponent, opponent, neutral, expert, devil's advocate
4. **Conflict Strategies**: Acknowledge, explore, vote, or defer
5. **Synthesis**: Combining perspectives into conclusions
6. **Statistics**: Track participation and cross-references

### Next Steps

- **Scenario 7**: Evaluation and Prompt Evolution
- Integrate real LLM agents into discussions
- Build more sophisticated conflict resolution